## 2021: Week 10 - Pokémon Hierarchies

I can hear the hesitancy already surrounding this challenge. "I know nothing about Pokémon, how can I participate?" Well, rest assured, you don't need to understand the data in order to work with it and complete this challenge. In fact, I believe this challenge could help with most hierarchical data. 

The aim of this challenge is to group Pokémon into Evolution Groups. Why? So that when we select any Pokémon, we can see all the information about what they evolved from, what they will evolve to etc. In the example below, Bulbasaur evolves into Ivysaur, who then evolves into Venusaur. Choose another Pokémon like Flareon and we can see it evolves from Eevee, who has multiple evolution possibilities.

How is this applicable to other hierarchical data? Well imagine that a Pokémon that has a null in the Evolves From field is similar to an employee at Work Level 1 in an organisation. Their manager would then sit in the Evolves To field. Our Evolution Group in this instance would be the Team the employee belongs to and it would give us the ability to type in any employee name and return information about which team they belong to and the structure of that team.

Sure, I could have chosen to design this challenge to be working with employees, but I believe it's useful to work with data outside of your comfort zone every now and then to prove to yourself that your prep skills are applicable no matter what the dataset. Plus it's just more fun to work with Pokémon!

### Input
We have 2 inputs this week:
1. Pokemon
![img](https://1.bp.blogspot.com/-6yOamFdTm1o/YCVAm0FbdtI/AAAAAAAAAvY/zQNLQy8hjAAiGj0Od3q1d1b27Mxnkml8ACLcBGAsYHQ/w400-h188/2021W10%2BInput1.png)

2. Evolution
![img2](https://1.bp.blogspot.com/-G9styqz2ZcI/YCVAaVD6E9I/AAAAAAAAAvU/O7pXXAZJsnQ3uXp2sgkSsUW8py1p6GieACLcBGAsYHQ/s320/2021W10%2BInput2.png)

### Requirment

- Input the data
- Our Pokémon dataset actually contains too many Pokémon: 
    - We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
    - This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega"
- Some Pokémon have more than one Type. We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon 
- Now we want to bring in information about what our Pokémon evolve to 
    - Warning!  In our Evolution dataset, we still have Pokémon beyond Gen III. You'll need to filter these out too, from both the evolved from and evolved to fields 
- Bring in information about what a Pokémon evolves from 
    - Ensure that we have all 386 of our Pokémon, with nulls if they don't have a pre-evolved form or if they don't evolve
- Finally, for Pokémon that have 3 evolutions, we want to know what the First Evolution is in their Evolution Group
- Some duplication may have occurred with all our joins, ensure no 2 rows are exactly the same
- Create a calculation for our Evolution Group
    - The Evolution Group will be named after the First Evolution e.g. in the above example, Bulbasaur is the name of the Evolution Group
- Output the data

### Output
![img3](https://1.bp.blogspot.com/-GXFwV1UpiX4/YCVijXraHSI/AAAAAAAAAvs/b9oxjFsWaucSvhHncE64YWynxWiBIe9tACLcBGAsYHQ/w640-h158/2021W%25600%2BOujt.png)

In [297]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [298]:
data = pd.read_excel("./data/Pokemon Input.xlsx", sheet_name=["Pokemon", "Evolution"], dtype="object")

In [299]:
pokemon = data["Pokemon"].copy()
evolution = data["Evolution"].copy()

In [300]:
# Our Pokémon dataset actually contains too many Pokémon
# We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
# This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega"
pokemon = pokemon.rename(columns={"#": "Number"})
pokemon.Number = pokemon.Number.astype(float).astype(int)
pokemon = pokemon.sort_values(by="Number")
pokemon = pokemon.loc[(pokemon["Number"] <= 386)]

mega = pokemon["Name"].map(lambda x: x.startswith("Mega"))
pokemon = pokemon.loc[~mega, :]
pokemon

,Number,Name,Type,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed
0,1,Bulbasaur,GRASS,318,45,49,49,65,65,45
1,1,Bulbasaur,POISON,318,45,49,49,65,65,45
2,2,Ivysaur,GRASS,405,60,62,63,80,80,60
3,2,Ivysaur,POISON,405,60,62,63,80,80,60
4,3,Venusaur,GRASS,525,80,82,83,100,100,80
...,...,...,...,...,...,...,...,...,...,...
614,385,Jirachi,STEEL,600,100,100,100,100,100,100
618,386,Deoxys- Defense Forme,PSYCHIC,600,50,70,160,70,160,90
619,386,Deoxys- Speed Forme,PSYCHIC,600,50,95,90,95,90,180
616,386,Deoxys- Normal Forme,PSYCHIC,600,50,150,50,150,50,150


In [301]:
# Some Pokémon have more than one Type. 
# We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon
pokemon = pokemon.drop_duplicates(subset=["Number"])
pokemon

,Number,Name,Type,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed
0,1,Bulbasaur,GRASS,318,45,49,49,65,65,45
2,2,Ivysaur,GRASS,405,60,62,63,80,80,60
4,3,Venusaur,GRASS,525,80,82,83,100,100,80
8,4,Charmander,FIRE,309,39,52,43,60,50,65
9,5,Charmeleon,FIRE,405,58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...,...
610,382,Kyogre,WATER,670,100,100,90,150,140,90
611,383,Groudon,GROUND,670,100,150,140,100,90,90
612,384,Rayquaza,DRAGON,680,105,150,90,150,90,95
615,385,Jirachi,PSYCHIC,600,100,100,100,100,100,100


In [302]:
# Now we want to bring in information about what our Pokémon evolve to
# Warning!  In our Evolution dataset, we still have Pokémon beyond Gen III. 
# You'll need to filter these out too, from both the evolved from and evolved to fields
evolution = evolution.loc[(evolution["Evolving from"].isin(pokemon["Name"])) | 
                          (evolution["Evolving to"].isin(pokemon["Name"]))]
evolution = evolution.drop_duplicates()
evolution

,Evolving from,Evolving to,Level,Condition,Evolution Type
0,Bulbasaur,Ivysaur,16,NaN,Level
1,Ivysaur,Venusaur,32,NaN,Level
2,Charmander,Charmeleon,16,NaN,Level
3,Charmeleon,Charizard,36,NaN,Level
4,Squirtle,Wartortle,16,NaN,Level
...,...,...,...,...,...
360,Sunkern,Sunflora,NaN,Sun Stone,Stone
361,Gloom,Bellossom,NaN,Sun Stone,Stone
363,Eevee,Jolteon,NaN,Thunderstone,Stone
364,Pikachu,Raichu,NaN,Thunderstone,Stone


In [303]:
# Bring in information about what a Pokémon evolves from
# Ensure that we have all 386 of our Pokémon, with nulls if they don't have a pre-evolved form or if they don't evolve
pokemon = pokemon.merge(evolution, how="left", left_on="Name", right_on="Evolving from")
pokemon

,Number,Name,Type,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed,Evolving from,Evolving to,Level,Condition,Evolution Type
0,1,Bulbasaur,GRASS,318,45,49,49,65,65,45,Bulbasaur,Ivysaur,16,NaN,Level
1,2,Ivysaur,GRASS,405,60,62,63,80,80,60,Ivysaur,Venusaur,32,NaN,Level
2,3,Venusaur,GRASS,525,80,82,83,100,100,80,NaN,NaN,NaN,NaN,NaN
3,4,Charmander,FIRE,309,39,52,43,60,50,65,Charmander,Charmeleon,16,NaN,Level
4,5,Charmeleon,FIRE,405,58,64,58,80,65,80,Charmeleon,Charizard,36,NaN,Level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,382,Kyogre,WATER,670,100,100,90,150,140,90,NaN,NaN,NaN,NaN,NaN
394,383,Groudon,GROUND,670,100,150,140,100,90,90,NaN,NaN,NaN,NaN,NaN
395,384,Rayquaza,DRAGON,680,105,150,90,150,90,95,NaN,NaN,NaN,NaN,NaN
396,385,Jirachi,PSYCHIC,600,100,100,100,100,100,100,NaN,NaN,NaN,NaN,NaN


In [304]:
# Finally, for Pokémon that have 3 evolutions, we want to know what the First Evolution is in their Evolution Group


In [305]:
tmp = pd.DataFrame(evolution["Evolving to"]).rename(columns={"Evolving to": "Evolving name"})
tmp["Evolving Group"] = evolution["Evolving from"]
tmp

,Evolving name,Evolving Group
0,Ivysaur,Bulbasaur
1,Venusaur,Ivysaur
2,Charmeleon,Charmander
3,Charizard,Charmeleon
4,Wartortle,Squirtle
...,...,...
360,Sunflora,Sunkern
361,Bellossom,Gloom
363,Jolteon,Eevee
364,Raichu,Pikachu


In [306]:
pokemon = pokemon.merge(tmp, how="left", left_on="Evolving from", right_on="Evolving name").drop("Evolving name", axis=1)
pokemon

,Number,Name,Type,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed,Evolving from,Evolving to,Level,Condition,Evolution Type,Evolving Group
0,1,Bulbasaur,GRASS,318,45,49,49,65,65,45,Bulbasaur,Ivysaur,16,NaN,Level,NaN
1,2,Ivysaur,GRASS,405,60,62,63,80,80,60,Ivysaur,Venusaur,32,NaN,Level,Bulbasaur
2,3,Venusaur,GRASS,525,80,82,83,100,100,80,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Charmander,FIRE,309,39,52,43,60,50,65,Charmander,Charmeleon,16,NaN,Level,NaN
4,5,Charmeleon,FIRE,405,58,64,58,80,65,80,Charmeleon,Charizard,36,NaN,Level,Charmander
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,382,Kyogre,WATER,670,100,100,90,150,140,90,NaN,NaN,NaN,NaN,NaN,NaN
394,383,Groudon,GROUND,670,100,150,140,100,90,90,NaN,NaN,NaN,NaN,NaN,NaN
395,384,Rayquaza,DRAGON,680,105,150,90,150,90,95,NaN,NaN,NaN,NaN,NaN,NaN
396,385,Jirachi,PSYCHIC,600,100,100,100,100,100,100,NaN,NaN,NaN,NaN,NaN,NaN


In [336]:
pokemon["First initial"] = pokemon["Name"].map(lambda x: x[:5])
pokemon["Last initial"] = pokemon["Name"].map(lambda x: x[-4:])

test = pokemon.groupby(["First initial"]).nunique()
first_initial = test[test["Number"] == 3].reset_index()["First initial"]

test = pokemon.groupby(["Last initial"]).nunique()
last_initial = test[test["Number"] == 3].reset_index()["Last initial"]

In [340]:
pokemon[pokemon["Last initial"].isin(last_initial)]

,Number,Name,Type,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed,Evolving from,Evolving to,Level,Condition,Evolution Type,Evolving Group,First initial,Last initial
0,1,Bulbasaur,GRASS,318,45,49,49,65,65,45,Bulbasaur,Ivysaur,16,NaN,Level,NaN,Bulba,saur
1,2,Ivysaur,GRASS,405,60,62,63,80,80,60,Ivysaur,Venusaur,32,NaN,Level,Bulbasaur,Ivysa,saur
2,3,Venusaur,GRASS,525,80,82,83,100,100,80,NaN,NaN,NaN,NaN,NaN,NaN,Venus,saur
43,44,Gloom,POISON,395,60,65,70,85,75,40,Gloom,Vileplume,NaN,Leaf Stone,Stone,Oddish,Gloom,loom
44,44,Gloom,POISON,395,60,65,70,85,75,40,Gloom,Bellossom,NaN,Sun Stone,Stone,Oddish,Gloom,loom
140,134,Vaporeon,WATER,525,130,65,60,110,95,65,NaN,NaN,NaN,NaN,NaN,NaN,Vapor,reon
142,136,Flareon,FIRE,525,65,130,60,95,110,65,NaN,NaN,NaN,NaN,NaN,NaN,Flare,reon
193,188,Skiploom,GRASS,340,55,45,50,45,65,80,Skiploom,Jumpluff,27,NaN,Level,Hoppip,Skipl,loom
202,197,Umbreon,DARK,525,95,65,110,60,130,65,NaN,NaN,NaN,NaN,NaN,NaN,Umbre,reon
253,246,Larvitar,ROCK,300,50,64,50,45,50,41,Larvitar,Pupitar,30,NaN,Level,NaN,Larvi,itar


In [ ]:
pokemon = pokemon.drop_duplicates(subset="Name")
pokemon

In [279]:
evolution.melt(id_vars="Evolving from", value_vars="Evolving to")

,Evolving from,variable,value
0,Bulbasaur,Evolving to,Ivysaur
1,Ivysaur,Evolving to,Venusaur
2,Charmander,Evolving to,Charmeleon
3,Charmeleon,Evolving to,Charizard
4,Squirtle,Evolving to,Wartortle
...,...,...,...
190,Sunkern,Evolving to,Sunflora
191,Gloom,Evolving to,Bellossom
192,Eevee,Evolving to,Jolteon
193,Pikachu,Evolving to,Raichu


,Evolving from,Evolving to,Level,Condition,Evolution Type
0,Bulbasaur,Ivysaur,16,NaN,Level
1,Ivysaur,Venusaur,32,NaN,Level
2,Charmander,Charmeleon,16,NaN,Level
3,Charmeleon,Charizard,36,NaN,Level
4,Squirtle,Wartortle,16,NaN,Level
...,...,...,...,...,...
360,Sunkern,Sunflora,NaN,Sun Stone,Stone
361,Gloom,Bellossom,NaN,Sun Stone,Stone
363,Eevee,Jolteon,NaN,Thunderstone,Stone
364,Pikachu,Raichu,NaN,Thunderstone,Stone
